In [1]:
import pandas as pd
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import datetime

from random import randint
from numpy import array
from numpy import argmax
from random import randint

from math import sin
from math import pi
from math import exp
from random import randint
from random import uniform

from numpy import array
from math import ceil
from math import log10

from random import random
from numpy import cumsum
from numpy import array_equal

import json
import pickle

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import MinMaxScaler

import re

pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 500)

In [2]:
path = './Datasets/'

In [3]:
log_df = pd.read_parquet(path + "logs.parquet")

In [4]:
log_df.head(10)

,resp_time,bytes_sent,resp_code,url,datetime,app_1_containers,app_2_containers,app_3_containers,app_4_containers
0,0.031,197,200,/2/4-3/65,2019-09-05 22:37:10,3,3,3,3
1,0.380,198,200,/3/3-4/37,2019-09-05 22:37:10,3,3,3,3
2,0.298,188,200,/4/3/188,2019-09-05 22:37:11,3,3,3,3
3,0.009,188,200,/3/3/180,2019-09-05 22:37:11,3,3,3,3
4,0.331,199,200,/3/3-4/115,2019-09-05 22:37:11,3,3,3,3
5,0.148,229,200,/3/3-3-3-3-3/101,2019-09-05 22:37:11,3,3,3,3
6,0.023,188,200,/3/3/180,2019-09-05 22:37:11,3,3,3,3
7,0.298,189,200,/4/3/188,2019-09-05 22:37:11,3,3,3,3
8,0.027,207,200,/3/4-2-2/38,2019-09-05 22:37:11,3,3,3,3
9,0.352,210,200,/3/4-4-3/60,2019-09-05 22:37:11,3,3,3,3


In [5]:
# IMPORTANT!! - deleting all logs without a response time (errors) - this loses key information but should simplify the problem while preserving the basic relationship

log_df.dropna(inplace=True)

In [6]:
# Feature transformations

# Convert resp_time and bytes_setn into float

log_df.resp_time = log_df.resp_time.astype("float")
log_df.bytes_sent = log_df.bytes_sent.astype("float")

In [7]:
app_cols = [col for col in log_df.columns if "app_" in col]

In [8]:
log_df

,resp_time,bytes_sent,resp_code,url,datetime,app_1_containers,app_2_containers,app_3_containers,app_4_containers
0,0.031,197.0,200,/2/4-3/65,2019-09-05 22:37:10,3,3,3,3
1,0.380,198.0,200,/3/3-4/37,2019-09-05 22:37:10,3,3,3,3
2,0.298,188.0,200,/4/3/188,2019-09-05 22:37:11,3,3,3,3
3,0.009,188.0,200,/3/3/180,2019-09-05 22:37:11,3,3,3,3
4,0.331,199.0,200,/3/3-4/115,2019-09-05 22:37:11,3,3,3,3
...,...,...,...,...,...,...,...,...,...
4483532,0.375,188.0,200,/4/4/223,2019-09-11 12:31:41,2,6,2,6
4483533,10.613,219.0,200,/3/3-3-3-4/9,2019-09-11 12:31:41,2,6,2,6
4483534,0.649,189.0,200,/4/3/169,2019-09-11 12:31:41,2,6,2,6
4483535,22.602,220.0,200,/4/4-4-3-4/16,2019-09-11 12:31:41,2,6,2,6


In [9]:
# Scale key cols between 0 and 1

log_df["resp_time"] = MinMaxScaler().fit_transform(log_df[["resp_time"]])
log_df[app_cols] = MinMaxScaler().fit_transform(log_df[app_cols])

In [10]:
log_df.drop(columns=["bytes_sent", "resp_code", "url", "datetime"], inplace=True)

In [11]:
log_df

,resp_time,app_1_containers,app_2_containers,app_3_containers,app_4_containers
0,0.000335,1.0,0.4,0.4,0.4
1,0.004109,1.0,0.4,0.4,0.4
2,0.003223,1.0,0.4,0.4,0.4
3,0.000097,1.0,0.4,0.4,0.4
4,0.003579,1.0,0.4,0.4,0.4
...,...,...,...,...,...
4483532,0.004055,0.5,1.0,0.2,1.0
4483533,0.114769,0.5,1.0,0.2,1.0
4483534,0.007018,0.5,1.0,0.2,1.0
4483535,0.244417,0.5,1.0,0.2,1.0


In [12]:
# Extract and prepare data

y = log_df.resp_time

X = log_df.drop(columns=["resp_time"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

X_train = np.ascontiguousarray(X_train)
y_train = np.ascontiguousarray(y_train)

X_test = np.ascontiguousarray(X_test)
y_test = np.ascontiguousarray(y_test)

In [13]:
# Linear regression

model = LinearRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

models_dict = {
    'Mean Absolute Error': mean_absolute_error(y_test, y_pred),
    'Mean Squared Error': mean_squared_error(y_test, y_pred),
    'Root Mean Squared Error': np.sqrt(mean_squared_error(y_test, y_pred)),
    "Datetime": "_".join(str(datetime.datetime.now()).split()),
    "Coefficients": model.coef_
}

print(models_dict)

{'Mean Absolute Error': 0.12479636821890362, 'Mean Squared Error': 0.02930586520524632, 'Root Mean Squared Error': 0.1711895592763949, 'Datetime': '2019-09-27_10:20:10.222345', 'Coefficients': array([ 0.00951008,  0.02049003, -0.10923875, -0.0532386 ])}


In [14]:
for k, v in models_dict.items():
    print(k)
    print(v)

Mean Absolute Error
0.12479636821890362
Mean Squared Error
0.02930586520524632
Root Mean Squared Error
0.1711895592763949
Datetime
2019-09-27_10:20:10.222345
Coefficients
[ 0.00951008  0.02049003 -0.10923875 -0.0532386 ]


In [15]:
# Random forest

model = RandomForestRegressor(max_depth=5, max_features="sqrt",  n_jobs=-1, n_estimators= 300)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

models_dict = {
    'Mean Absolute Error': mean_absolute_error(y_test, y_pred),
    'Mean Squared Error': mean_squared_error(y_test, y_pred),
    'Root Mean Squared Error': np.sqrt(mean_squared_error(y_test, y_pred)),
    "Datetime": "_".join(str(datetime.datetime.now()).split()),
    "Feature importances": model.feature_importances_
}

print(models_dict)

{'Mean Absolute Error': 0.12336268349548564, 'Mean Squared Error': 0.028540787954385374, 'Root Mean Squared Error': 0.16894019046510328, 'Datetime': '2019-09-27_10:21:39.100471', 'Feature importances': array([0.01526595, 0.1167221 , 0.8048925 , 0.06311945])}


In [16]:
for k, v in models_dict.items():
    print(k)
    print(v)

Mean Absolute Error
0.12336268349548564
Mean Squared Error
0.028540787954385374
Root Mean Squared Error
0.16894019046510328
Datetime
2019-09-27_10:21:39.100471
Feature importances
[0.01526595 0.1167221  0.8048925  0.06311945]
